In [1]:
import pandas as pd

import torch
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer, util

DOLLY_MODEL = "databricks/dolly-v2-3b"
EMBEDDING_MODEL = "all-mpnet-base-v2"

USE_DOLLY_FOR_EMBEDDING = False


In [14]:
generate_text = pipeline(model=DOLLY_MODEL, torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True, do_sample=False)
# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)



In [3]:

if USE_DOLLY_FOR_EMBEDDING:
    embedding_model = SentenceTransformer(DOLLY_MODEL)
else:
    embedding_model = SentenceTransformer(EMBEDDING_MODEL)

In [4]:
df_text_chunks = pd.read_feather("../data/paper_extracts_embed.feature")
df_text_chunks.head()


,text_chunk,title,embeddings,embeddings_dolly
0,Texture Synthesis Using Convolutional NeuralNe...,Texture Synthesis Using Convolutional Neural N...,"[0.01643215, 0.08131, -0.051866785, 0.07214568...","[-0.4477233, 0.7085075, 0.9448252, -0.6605596,..."
1,2 Convolutional neural network We use the VGG...,Texture Synthesis Using Convolutional Neural N...,"[0.00494202, -0.0018913345, -6.29354e-05, 0.09...","[-0.16315894, 1.1724734, 0.34991983, -0.975609..."
2,3 different features. These feature correlati...,Texture Synthesis Using Convolutional Neural N...,"[0.040369663, 0.036883876, -0.026579408, 0.090...","[-0.5808077, 0.93010116, 0.35125256, -0.081901..."
3,conv1_1pool1pool2pool3pool4originalPortilla &...,Texture Synthesis Using Convolutional Neural N...,"[-0.005054468, -0.011686467, -0.05832806, 0.06...","[-1.1286112, 0.94930226, 0.83858913, -0.496184..."
4,6 Classification performance 1.00.80.60.4 top...,Texture Synthesis Using Convolutional Neural N...,"[0.02261693, -0.04751408, -0.008524225, 0.0593...","[-0.7846361, -0.06361116, 1.3205526, 1.9582235..."


In [22]:


def get_similarity(embedding_model, query, doc_emb, docs):
    query_emb = embedding_model.encode(query)
    scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
    #Combine docs & scores
    doc_score_pairs = list(zip(docs, scores))
    #Sort by decreasing score
    doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    return doc_score_pairs






In [32]:
if USE_DOLLY_FOR_EMBEDDING:
    doc_emb = df_text_chunks["embeddings_dolly"]
else:
    doc_emb = df_text_chunks["embeddings"]

def get_context(query, docs = df_text_chunks["text_chunk"], doc_emb=doc_emb, embedding_model=embedding_model):
    doc_score_pairs = get_similarity(embedding_model, query, doc_emb, docs)
    context_chunks = "\n\n".join([doc_score_pairs[i][0] for i in [1]])
    return context_chunks


query = ["What is the goal of visual texture synthesis?", "what is VGG-19", "What are Gram matrices used for"][2]
context = get_context(query=query)

print("\n----------------With Context--------------------\n")
print(llm_context_chain.predict(instruction=query, context=context_chunks).lstrip())

print("\n----------------Without Context--------------------\n")
print(llm_context_chain.predict(instruction=query, context="").lstrip())


----------------With Context--------------------

Gram matrices are used to measure the similarity between feature responses of different layers of a convolutional neural network. They are used in the context of texture synthesis to find a generating process that maps a source texture to another.

----------------Without Context--------------------

A Gram matrix is a type of matrix that is used in linear algebra to represent a inner product between vectors.  The inner product is computed from a set of vectors, called a basis, and a matrix that represents the basis.  The Gram matrix is used to represent the linear relationship between the basis vectors.  For example, the inner product of two vectors can be represented by the dot product of the corresponding vectors.  This dot product is computed by first applying the corresponding basis matrix to the two vectors.  The result of this computation is a number.  This number is called the inner product of the two vectors.  The inner produc

In [33]:
context

'3 \x0cdifferent features. These feature correlations are, up to a constant of proportionality, given by theGram matrix Gl  RNl Nl , where Glij is the inner product between feature map i and j in layer l:XllGlij =FikFjk.(1)k1 2 L A set of Gram matrices {G , G , ..., G } from some layers 1, . . . , L in the network in response toa given texture provides a stationary description of the texture, which fully specifies a texture in ourmodel (Fig. 1A). 4 Texture generation To generate a new texture on the basis of a given image, we use gradient descent from a white noiseimage to find another image that matches the Gram-matrix representation of the original image.This optimisation is done by minimising the mean-squared distance between the entries of the Grammatrix of the original image and the Gram matrix of the image being generated (Fig. 1B). be the original image and the image that is generated, and Gl and Gl their respectiveLet ~x and ~xGram-matrix representations in layer l (Eq. 1). The

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


A Gram matrix is a type of matrix that is used in linear algebra to represent a inner product between vectors.  The inner product is computed from a set of vectors, called a basis, and a matrix that represents the basis.  The Gram matrix is used to represent the linear relationship between the basis vectors.  For example, the inner product of two vectors can be represented by the dot product of the corresponding vectors.  This dot product is computed by first applying the corresponding basis matrix to the two vectors.  The result of this computation is a number.  This number is called the inner product of the two vectors.  The inner product of two vectors can also be represented by a dot product of the two Gram matrices of the basis vectors.  The Gram matrix of a basis is a square matrix with the same dimensions as the basis.  The inner product of two vectors is the dot product of the corresponding vectors and the basis.  The inner product of two vectors can also be represented by a do